# Block 6 Exercise 2: finding the best parameters for predicting the fare of taxi rides
We return to our Random Forest Regression and want to automatically optimize all free parameters ...

In [26]:
import pandas as pd
import numpy as np
import folium


In [27]:
#check if notebook runs in colab
import sys
IN_COLAB = 'google.colab' in sys.modules
print('running in Colab:',IN_COLAB)
path='..'
if IN_COLAB:
  #in colab, we need to clone the data from the repo
  !git clone https://github.com/keuperj/DataScienceSS20.git
  path='DataScienceSS20'

running in Colab: False


In [28]:
# we load the data we have saved after wrangling and pre-processing in block I
X=pd.read_csv(path+'/DATA/train_cleaned.csv')
drop_columns=['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1','key','pickup_datetime','pickup_date','pickup_latitude_round3','pickup_longitude_round3','dropoff_latitude_round3','dropoff_longitude_round3']
X=X.drop(drop_columns,axis=1)
X=pd.get_dummies(X)# one hot coding
#generate labels
y=X['fare_amount']
X=X.drop(['fare_amount'],axis=1)

### Scikit Optimize
Scikit Optimize (https://scikit-optimize.github.io/stable/index.html) is a AutoML toolbox wrapped around Scikit-Learn. It allows us to use state-of-the-art automatic hyper-parameter optimization on top of our learning algorithms.   



In [29]:
# install 
!pip install scikit-optimize

### E 2.1 Bayesian Optimization of a Random Forest Regression Model
use Bayesian Optimization with Cross-Validation (https://scikit-optimize.github.io/stable/modules/generated/skopt.BayesSearchCV.html#skopt.BayesSearchCV) to find the best regression model. Compare
* linear regression (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression) 
* Random Forest regression (https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor)
* and SVM regression (https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR)

NOTES: this can become quite compute intensive! Hence,
* use a smaller subset of the training data to run the experiments 
* think about the range of your parameters (e.g. larger number of trees in RF or high C-values in SMV will make models expensive)
* optimize only the following parameters per model type:
    * linear: no parameters to optimize
    * RF: #trees and depth
    * SVM: C and gamma (use RBF kernel)
* parallelize -> n_jobs
* use CoLab to rum the job for up to 12h 


In [30]:
from sklearn.model_selection import train_test_split
subset_size=1000
X_train, X_test, y_train, y_test = train_test_split(X.to_numpy()[:subset_size,:], y.to_numpy()[:subset_size])

In [31]:
from skopt import BayesSearchCV
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from skopt.space import Real, Integer

In [32]:
linear = LinearRegression()
linear.fit(X_train,y_train)
print("test score:", linear.score(X_test, y_test))

test score: 0.7336802942538188


In [33]:
optRFR = BayesSearchCV(
    RandomForestRegressor(),
    {
        'n_estimators':Integer(10,200, prior='log-uniform'),
        'max_depth':Integer(2,20,prior='log-uniform')       
    },
    n_iter=32,
    n_jobs=-1,
    random_state=0
)
optRFR.fit(X_train,y_train)
print("best validation score:", optRFR.best_score_)
print("best parameters:", optRFR.best_params_)
print("test score:", optRFR.score(X_test, y_test))

C:\Dev\anaconda\lib\site-packages\skopt\optimizer\optimizer.py:409: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Dev\anaconda\lib\site-packages\skopt\optimizer\optimizer.py:409: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Dev\anaconda\lib\site-packages\skopt\optimizer\optimizer.py:409: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


best validation score: 0.7304158778058502
best parameters: OrderedDict([('max_depth', 19), ('n_estimators', 154)])
test score: 0.7738044529421964


In [34]:
optSVR = BayesSearchCV(
    SVR(),
    {
        'C': Real(1e-6, 1e+6, prior='log-uniform'),
        'gamma': Real(1e-6, 1e+1, prior='log-uniform'),         
    },
    n_iter=32,
    n_jobs=-1,
    random_state=0
)
optSVR.fit(X_train,y_train)
print("best validation score:", optSVR.best_score_)
print("best parameters:", optSVR.best_params_)
print("test score:", optSVR.score(X_test, y_test))

best validation score: 0.7445135744340643
best parameters: OrderedDict([('C', 603.5578367071575), ('gamma', 0.000648014134267499)])
test score: 0.732207308993835
